# __Plotting__
Purpose:  Investigation of (price and patent) data to determine points of interest

In [5]:
import pandas as pd
import dill
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
# Updated with new post-merge dataset!

# Unpickle Price_Patent_Data from the 'MergingAllData' Jupyter Notebook
Price_Patent_Data = dill.load(open('Price_Patent_Data.pkd', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'Price_Patent_Data.pkd'

In [ ]:
Price_Patent_Data.info()

In [ ]:
#Convert all to datetimes
Price_Patent_Data['effective_date'] = pd.to_datetime(Price_Patent_Data['effective_date']) #, format='%Y/%m/%d')
Price_Patent_Data['as_of_date'] = pd.to_datetime(Price_Patent_Data['as_of_date']) #, format='%Y/%m/%d')
Price_Patent_Data['corresponding_generic_drug_effective_date'] = pd.to_datetime(Price_Patent_Data['corresponding_generic_drug_effective_date']) #, format='%Y/%m/%d')
Price_Patent_Data['approval_date'] = pd.to_datetime(Price_Patent_Data['approval_date']) #, format='%Y/%m/%d')
Price_Patent_Data['patent_expire_date_text'] = pd.to_datetime(Price_Patent_Data['patent_expire_date_text']) #, format='%Y/%m/%d')
Price_Patent_Data['submission_date'] = pd.to_datetime(Price_Patent_Data['submission_date']) #, format='%Y/%m/%d')
Price_Patent_Data['exclusivity_date'] = pd.to_datetime(Price_Patent_Data['exclusivity_date']) #, format='%Y/%m/%d')

In [ ]:
Price_Patent_Data.reset_index(inplace = True)

In [ ]:
# Group drugs by name and sort values by price (highest to lowest)
price_table = Price_Patent_Data.groupby('ndc_description').mean().sort_values(by='nadac_per_unit', ascending = False)
price_table.head(10)

In [ ]:
# Group drugs by name and sort values by the price of the corresponding generic drug (highest to lowest)
price_table = Price_Patent_Data[Price_Patent_Data['classification_for_rate_setting']=='G'].groupby(Price_Patent_Data['ndc_description'])
# price_table = price_table.sort_values('nadac_per_unit', ascending = False)
price_table.head(10)

In [ ]:
Price_Patent_Data.info(verbose = True, null_counts = True)

In [ ]:
# Top 15 drugs (brand and generic) by frequency in the dataset
top_drugs_by_freq = Price_Patent_Data['ndc_description'].value_counts().sort_values(ascending = False)
top_15_drugs_by_freq = top_drugs_by_freq[:15].index.tolist()
top_15_drugs_by_freq

In [ ]:
# Top 15 generic drugs by frequency in the dataset
top_drugs_by_freq = Price_Patent_Data[Price_Patent_Data['classification_for_rate_setting']=='G']
top_generic_drugs_by_freq = top_drugs_by_freq['ndc_description'].value_counts().sort_values(ascending = False)
top_15_drugs_generic = top_generic_drugs_by_freq[:15].index.tolist()
top_15_drugs_generic

In [ ]:
# Are the top 15 drugs by frequency all generics?
top_15_drugs_by_freq == top_15_drugs_generic

In [ ]:
# Top 15 brand-name drugs by frequency in the dataset
top_brand_drugs_by_freq = Price_Patent_Data[Price_Patent_Data['classification_for_rate_setting']=='B']
top_brand_drugs_sorted = top_brand_drugs_by_freq['ndc_description'].value_counts().sort_values(ascending = False)
top_15_drugs_brand = top_brand_drugs_sorted[:15].index.tolist()
top_15_drugs_brand

In [ ]:
# Plot prices for the top 15 generic drugs over time
top_15_df = Price_Patent_Data[Price_Patent_Data['ndc_description'].isin(top_15_drugs_generic)]

fig = plt.figure(figsize = (12, 9))
sns.lineplot(x = 'effective_date', 
             y = 'nadac_per_unit',
             hue = 'ndc_description',
             data = top_15_df).set_title('Top 15 generic drugs (by frequency) prices over time')

In [ ]:
# Top 15 brand-name drugs without generic equivalents by frequency in the dataset
brand_drugs_by_freq = Price_Patent_Data[(Price_Patent_Data['classification_for_rate_setting']=='B')]
brand_drugs_no_te = brand_drugs_by_freq[brand_drugs_by_freq['te_code'].isnull()] # Roughly: brand drugs with no therapeutic equivalents


sorted_brand_only_drugs = brand_drugs_no_te['ndc_description'].value_counts().sort_values(ascending = False)
top_15_brand_only_drugs = sorted_brand_only_drugs[:15].index.tolist()

# # Plot prices for the top 15 brand-name drugs over time
top_15_brand_only_drugs_no_te = Price_Patent_Data[Price_Patent_Data['ndc_description'].isin(top_15_brand_only_drugs)]

fig = plt.figure(figsize = (12, 9))
sns.lineplot(x = 'effective_date', 
             y = 'nadac_per_unit',
             hue = 'ndc_description',
             data = top_15_brand_only_drugs_no_te).set_title('Tope 15 brand-only drug prices (by frequency) over time')
# plt.title()
plt.legend(loc = 'center left', bbox_to_anchor = (1, 0.5), ncol=1)

-------------------------------------------
We'll drop Epclusa (for the purpose of these plots) since, with respect to price, it's an outlier.  We'll also see that it'll allow us to see the other drug price trends more accurately.

In [ ]:
# Plot prices for the top 14 (EPCLUSA excluded as an outlier) brand-name drugs over time
top_14_brand_only_drugs = top_15_brand_only_drugs_no_te[~top_15_brand_only_drugs_no_te['ndc_description'].str.contains('EPCLUSA')]

fig = plt.figure(figsize = (12, 9))
g = sns.lineplot(x = 'effective_date', 
             y = 'nadac_per_unit',
             hue = 'ndc_description',
             data = top_14_brand_only_drugs).set_title('Top 14 brand-only drug prices (by frequency) over time')
plt.legend(loc = 'center left', bbox_to_anchor = (1, 0.5), ncol=1)

### __Patent data over time__

In [ ]:
# Plot Patent approval dates, followed by expiry dates on a horizontal lollipop plot.  
# I believe that the green dots all represent extensions to that particular drug

fig = plt.figure(figsize=(12, 9))

plt.hlines(y=top_15_brand_only_drugs_no_te['ndc_description'], 
           xmin=top_15_brand_only_drugs_no_te['approval_date'], 
           xmax=top_15_brand_only_drugs_no_te['patent_expire_date_text'], 
           color='grey', 
           alpha=0.4)
plt.scatter(top_15_brand_only_drugs_no_te['approval_date'], 
            top_15_brand_only_drugs_no_te['ndc_description'], 
            color = 'skyblue', 
            alpha = 1, 
            label = 'approval_date')
plt.scatter(top_15_brand_only_drugs_no_te['patent_expire_date_text'],
            top_15_brand_only_drugs_no_te['ndc_description'],
            color = 'green',
            alpha = 0.4, 
            label = 'patent_expire_date')
plt.set_title('Patent approval and expiry dates of top 15 drugs (by frequency) over time')
plt.legend(loc = 'lower right')

### __Brand Drugs in Dataset__

In [ ]:
Price_Patent_Data['classification_for_rate_setting'].value_counts()

### __Drugs without Therapeutic Equivalents__

In [ ]:
top_15_brand_only_drugs

In [ ]:
#Order Drugs by highest price
#top_brand_drugs_by_freq ==> a variable created further up in the workflow
top_brand_only_drugs = top_brand_drugs_by_freq[top_brand_drugs_by_freq['te_code'].isnull()] #with no therapeutic equivalents
drugs_by_price = top_brand_only_drugs.sort_values(by='nadac_per_unit', ascending = False) #sort all values by price

In [ ]:
# Aggregate drugs on list by top 15 drug names
drugs_by_price_grouped = drugs_by_price.groupby('ndc_description')
drugs_by_price_grouped.apply(lambda drugs_by_price_grouped: drugs_by_price_grouped.sort_values(by = ['nadac_per_unit'], ascending = False)) #sort first by price, then get the top 15
drugs_by_price_grouped = drugs_by_price_grouped.head(15)
drugs_by_price_grouped = drugs_by_price_grouped['ndc_description'] # Select just the names that have been ordered by price
drugs_by_price_grouped = drugs_by_price_grouped.unique()[:15] #get unique names from the list

In [ ]:
# Plot prices for the top 5 brand-name drugs over time (use list to filter) - the top 5 are much higher than the rest
highest_price_15_brand_only_drugs = Price_Patent_Data[Price_Patent_Data['ndc_description'].isin(drugs_by_price_grouped[:5])]

fig = plt.figure(figsize = (12, 9))
sns.lineplot(x = 'effective_date', 
             y = 'nadac_per_unit',
             hue = 'ndc_description',
             data = highest_price_15_brand_only_drugs).set_title('Top 5 brand-only drug prices (by price) over time')
plt.legend(loc = 'center left', bbox_to_anchor = (1, 0.5), ncol=1)


In [ ]:
# Plot prices for the top 5-15 brand-name drugs over time (use list to filter)
highest_price_15_brand_only_drugs = Price_Patent_Data[Price_Patent_Data['ndc_description'].isin(drugs_by_price_grouped[5:])]

fig = plt.figure(figsize = (12, 9))
sns.lineplot(x = 'effective_date', 
             y = 'nadac_per_unit',
             hue = 'ndc_description',
             data = highest_price_15_brand_only_drugs).set_title('Top 5 to 15 brand-only drug prices (by price) over time')
plt.legend(loc = 'center left', bbox_to_anchor = (1, 0.5), ncol=1)


In [ ]:
###  TO DO:  Plot brands with generic equivalent on top of generic approval date
Price_Patent_Data.classification_for_rate_setting.value_counts()

In [ ]:
# Look at brand drugs with therapeutic equivalents
# consider looking specifically at these drugs when regressing for price

all_drugs_with_te = Price_Patent_Data[Price_Patent_Data['classification_for_rate_setting'].str.contains('B', na=False)] #filter for brand only drugs

brand_drugs_with_te = all_drugs_with_te[all_drugs_with_te['te_code'].notnull()] #with therapeutic equivalents
brand_drugs_with_te = brand_drugs_with_te.groupby('ndc_description')
brand_drugs_with_te.apply(lambda brand_drugs_with_te: brand_drugs_with_te.sort_values(by=['nadac_per_unit'], ascending = False)) #sort all values by price

brand_drugs_with_te = brand_drugs_with_te['ndc_description'].unique()[:15].index.to_list() #convert the top 15 unique elements to a list
brand_drugs_with_te

In [ ]:
# Plot prices for the top 5:15 brand-name drugs over time (use list to filter)
highest_price_15_drugs_with_te = Price_Patent_Data[Price_Patent_Data['ndc_description'].isin(brand_drugs_with_te)]

fig = plt.figure(figsize = (12, 9))
g = sns.lineplot(x = 'effective_date', 
             y = 'nadac_per_unit',
             hue = 'ndc_description',
             data = highest_price_15_drugs_with_te).set_title('Top 15 brand-name drug prices (with therapeutic equivalents) over time')
plt.legend(loc = 'center left', bbox_to_anchor = (1, 0.5), ncol=1)
g.set(ylim(0, 20))

In [ ]:
# Create a correlation (heatmap) plot
import numpy as np

#Binarizing variables first would allow a comparison of more variables, however, I get a memory error if I try pd.get_dummies()

corr = Price_Patent_Data.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize = (11, 9))
cmap = sns.diverging_palette(240, 10, as_cmap = True)
sns.heatmap(corr, mask = mask, cmap = cmap, vmax = 0.3, center = 0, square = True,  linewidths = 0.5, cbar_kws = {'shrink': 0.5})

## __Original Plots__

Plots pre-TDI

### __Ibuprofen Prices over Time__
In this case:
* time == effective_date
* price == nadac_per_unit

In [ ]:
# Create a dataframe for Ibuprofen 200mg tablets
ibuprofen_200mg_tab = Price_Patent_Data[Price_Patent_Data['ndc_description']=='IBUPROFEN 200MG TABLET']
g = sns.scatterplot(x = ibuprofen_200mg_tab['effective_date'], y = ibuprofen_200mg_tab['nadac_per_unit'])
plt.xlim('01/01/2016','01/01/2020')
plt.ylim(0.0225, 0.0375)
plt.xticks(rotation = 30)

### __Ibuprofen Prices v. 'as_of_date'__
This column is not in the data dictionary, but I believe it's a date corresponding to prices that's updated maybe more frequenly than the effective_date?

In [ ]:
g = sns.scatterplot(x = ibuprofen_200mg_tab['as_of_date'], y = ibuprofen_200mg_tab['nadac_per_unit'])
plt.xlim('01/01/2016','01/01/2020')
plt.ylim(0.0225, 0.0375)
plt.xticks(rotation=65)

### __Prices for all Ibuprofen in Dataset v. Time__

In [ ]:
ibuprofen_all = Price_Patent_Data[Price_Patent_Data['ingredient'].str.contains('IBUPROFEN', na=False)]
g = sns.scatterplot(x = 'effective_date', y = 'nadac_per_unit', hue = ibuprofen_all.ndc_description, data = ibuprofen_all)
# plt.xlim('01/01/2014','01/01/2020')
plt.ylim(0.02, 0.1)
plt.xlim('2016-01-01', '2021-01-01')
g.legend(loc = 'center left', bbox_to_anchor = (1, 0.5), ncol=1)


### __Cost of Metformin v. Time__

Multiple varieties of metformin displayed here

In [ ]:
metformin = Price_Patent_Data[Price_Patent_Data['ndc_description'].str.contains('METFORMIN', na = False)]
g = sns.scatterplot(x = metformin['effective_date'], y = metformin['nadac_per_unit'])
plt.xlim('01/01/2013','01/01/2020')

In [ ]:
# Varieties of metformin in dataset
metformin['ndc_description'].value_counts()

### __Price of Lisinopril over Time__
Multiple varieties of lisinopril displayed here

In [ ]:
Price_Patent_Data = Price_Patent_Data[Price_Patent_Data['ndc_description'].notnull()]
Price_Patent_Data['ndc_description'].isnull().value_counts(dropna = False)

In [ ]:
lisinopril = Price_Patent_Data[Price_Patent_Data['ndc_description'].str.contains('LISINOPRIL')]
g = sns.scatterplot(x = lisinopril['effective_date'], y = lisinopril['nadac_per_unit']).set(xlim=('01/01/2016','01/01/2020'))

### __Price of Prinivil over Time__
Multiple varieties of prinivil displayed here

In [ ]:
prinivil = Price_Patent_Data[Price_Patent_Data['ndc_description'].str.contains('PRINIVIL')]
g = sns.scatterplot(x = prinivil['effective_date'], y = prinivil['nadac_per_unit'])

### __Price of Simvastatin over Time__
Multiple varieties of simvastatin displayed here

In [ ]:
simvastatin  = Price_Patent_Data[Price_Patent_Data['ndc_description'].str.contains('SIMVASTATIN')]
g = sns.scatterplot(x = simvastatin['effective_date'], y = simvastatin['nadac_per_unit']).set(xlim=('01/01/2016','01/01/2020'))

In [ ]:
#Observations in dataset for 'SIMVASTATIN'
Price_Patent_Data['ndc_description'].str.contains('SIMVASTATIN').value_counts(dropna = False)

### __Price of Zocor over Time__
Multiple varieties of zocor displayed here

In [ ]:
zocor  = Price_Patent_Data[Price_Patent_Data['ndc_description'].str.contains('ZOCOR')]
g = sns.scatterplot(x = zocor['effective_date'], y = zocor['nadac_per_unit'])

In [ ]:
#Observations in dataset for 'ZOCOR'
Price_Patent_Data['ndc_description'].str.contains('ZOCOR').value_counts(dropna = False)

### __Price of Ambien over Time__
Multiple varieties of ambien displayed here

In [ ]:
# Price of Ambien over time (no generic equivalent)
ambien  = Price_Patent_Data[Price_Patent_Data['ndc_description'].str.contains('AMBIEN')]
sns.scatterplot(x = ambien['effective_date'], y = ambien['nadac_per_unit'], hue = ambien['classification_for_rate_setting']).set(xlim=('01/01/2016','01/01/2020'))

In [ ]:
sns.scatterplot(x = ambien['corresponding_generic_drug_effective_date'], y = ambien['corresponding_generic_drug_nadac_per_unit'])
plt.xlim('01/01/2014','01/01/2020')

In [ ]:
brands_with_generics = Price_Patent_Data[Price_Patent_Data['corresponding_generic_drug_nadac_per_unit'].notnull()]
brands_with_generics.head()

In [ ]:
brands_with_generics.info()

## __Brands with Generics__

### __Nicorette__

In [ ]:
#We see a very clear clustering here.  Look at labeling these drugs with different colors and a legend to better tell what's going on.
brands_with_generics_nicorette = brands_with_generics[Price_Patent_Data['ndc_description'].str.contains('NICORETTE')]
g = sns.scatterplot(x = brands_with_generics_nicorette['effective_date'], y = brands_with_generics_nicorette['corresponding_generic_drug_nadac_per_unit']).set(xlim=('01/01/2016','01/01/2020'))

#### __Ambien__

In [ ]:
ambien  = Price_Patent_Data[Price_Patent_Data['ndc_description'].str.contains('AMBIEN')]
sns.scatterplot(x = ambien['corresponding_generic_drug_effective_date'], y = ambien['corresponding_generic_drug_nadac_per_unit'])
sns.scatterplot(x = ambien['effective_date'], y = ambien['nadac_per_unit'])
plt.xlim('01/06/2016','01/01/2020')

In [ ]:
# Looking at more generic vs. brands
ambien_generic = Price_Patent_Data[Price_Patent_Data['ndc_description'].str.contains('ZOLPIDEM')]
ambien_generic['ndc_description'].value_counts()

In [ ]:
ambien_brand = Price_Patent_Data[Price_Patent_Data['ndc_description'].str.contains('AMBIEN')]
ambien_brand['ndc_description'].value_counts()

In [ ]:
ambien_patent = ambien_brand[ambien_brand['approval_date'].notnull()]
ambien_patent['approval_date'].head()

In [ ]:
sns.lineplot(x = 'effective_date', y = 'nadac_per_unit', data = ambien_generic)
sns.lineplot(x = 'effective_date', y = 'nadac_per_unit', data = ambien_brand)